In [3]:
import sqlalchemy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')

import dbconfig as db

engine = sqlalchemy.engine_from_config(db.config, prefix='db.')

# Feature generation

queries to pull out features to input to preprocessing

## Duration of Activity

In [14]:
doa_feat = """SELECT DISTINCT ON (exampleid)
  last_value(t_trace) OVER wnd - first_value(t_trace) OVER wnd AS doa,
  exampleid
 FROM raw.frontpage_traces
 WINDOW wnd AS (
   PARTITION BY exampleid ORDER BY t_trace
   ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
 );"""

In [15]:
df = pd.read_sql(doa_feat, engine)

## Total number cells exchanged

In [16]:
num_cells = """SELECT exampleid, count(distinct cellid) as num_cells
FROM raw.frontpage_traces
GROUP BY exampleid;"""

## Total number incoming and outgoing cells

In [17]:
num_incoming = """SELECT exampleid, count(distinct cellid) AS num_incoming
FROM raw.frontpage_traces
WHERE ingoing = 't'
GROUP BY exampleid;
"""

num_outgoing = """SELECT exampleid, count(distinct cellid) AS num_outgoing
FROM raw.frontpage_traces
WHERE ingoing = 'f'
GROUP BY exampleid;
"""

## Overall ratio incoming/outgoing cells

## Traffic Bursts

## Extract common sequences